Importing necessary packages

In [23]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

Load the intent Json file , Extract training sentences, labels, and responses from the JSON data

In [24]:
with open('intent.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intent']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

This where a label encoder is made. We then fit the training_labels and transform them

In [25]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

sets up a tokenizer to preprocess json data

In [26]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

text preprocessing to remove stopwords and punctuation marks

In [27]:
import nltk
nltk.download('stopwords')  # download the stopwords resource file

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))- {'a','an'}  # initialize the stopwords set

punctuations = ['.', ',', '!', '?', ':', ';', '(', ')', '[', ']', '{', '}', '\'', '\"', '`', '``', '\'\'', '-', '/', '\\']

tokens = ['This', 'is', 'an', 'example', 'sentence', 'showing', 'stop', 'word','what','human', 'removal', 'and', 'punctuation', 'removal', '.']


# remove stopwords and punctuations from the list of tokens
tokens = [token.lower() for token in tokens if token.lower() not in stop_words and token not in punctuations]

print(tokens)  # ['example', 'sentence', 'showing', 'stop', 'word', 'what', 'human', 'removal', 'punctuation', 'removal']

['an', 'example', 'sentence', 'showing', 'stop', 'word', 'human', 'removal', 'punctuation', 'removal']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Create a vocabulary

In [28]:
vocabulary = set(tokens)

Build the LSTM-based sequence-to-sequence model

In [29]:
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_len))
model.add(LSTM(64))
model.add(Dropout(0.5)) # add a dropout layer with a rate of 0.5
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Convert the training labels to one-hot vectors

In [30]:
one_hot_labels = tf.keras.utils.to_categorical(training_labels, num_classes)

Train the model

In [31]:
model.fit(padded_sequences, one_hot_labels, epochs=500)

Epoch 1/500
9/9 [==============================] - 3s 21ms/step - loss: 3.8060 - accuracy: 0.0370
Epoch 2/500
9/9 [==============================] - 0s 19ms/step - loss: 3.7965 - accuracy: 0.0630
Epoch 3/500
9/9 [==============================] - 0s 19ms/step - loss: 3.7854 - accuracy: 0.0926
Epoch 4/500
9/9 [==============================] - 0s 18ms/step - loss: 3.7699 - accuracy: 0.0889
Epoch 5/500
9/9 [==============================] - 0s 19ms/step - loss: 3.7455 - accuracy: 0.1111
Epoch 6/500
9/9 [==============================] - 0s 18ms/step - loss: 3.6996 - accuracy: 0.0889
Epoch 7/500
9/9 [==============================] - 0s 19ms/step - loss: 3.6358 - accuracy: 0.0926
Epoch 8/500
9/9 [==============================] - 0s 20ms/step - loss: 3.5329 - accuracy: 0.1556
Epoch 9/500
9/9 [==============================] - 0s 20ms/step - loss: 3.3562 - accuracy: 0.1407
Epoch 10/500
9/9 [==============================] - 0s 20ms/step - loss: 3.2008 - accuracy: 0.1963
Epoch 11/500
9/9 [=

# Save the trained model

In [32]:

model.save('chatbot_lstm.h5')

 install colorama library to add colored output to console text

In [20]:
pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


save the fitted tokenizer and label encoder using pickle.

In [33]:
import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


In [34]:
import re
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import colorama 
import random
import pickle

stop_words = set(stopwords.words('english'))- {'are', 'you','who','what','how'}
punctuations = ['.', ',', '!', '?', ':', ';', '(', ')', '[', ']', '{', '}', '\'', '\"', '`', '``', '\'\'', '-', '/', '\\']

def clean_up_sentence(sentence):
    # Remove punctuations and extra spaces
    sentence = re.sub(r'[^\w\s]','',sentence)
    sentence = re.sub('\s+',' ', sentence)

    # Convert to lower case and split into individual words
    words = sentence.lower().split()
    
    # Remove stop words
    words = [word for word in words if word not in stop_words and word not in punctuations]
    
    # Join the words back into a sentence
    return " ".join(words)

with open("intent.json") as file:
    data = json.load(file)

def chat():
    # load trained model
    model = keras.models.load_model('chatbot_lstm.h5')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(colorama.Fore.LIGHTBLUE_EX + "User: " + colorama.Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        # Clean up user input
        clean_input = clean_up_sentence(inp)

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([clean_input]),
                                             truncating='post', maxlen=max_len))[0]
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        #This creates a kind of fallback intent with an if else.
        if result[np.argmax(result)] > 0.7:
          for i in data['intent']:
              if i['tag'] == tag:
                  print(colorama.Fore.GREEN + "DiamondChatBot:" + colorama.Style.RESET_ALL , np.random.choice(i['responses']))
        else:
          print("I do not understand this. Could you please repeat or rephrase the question.")

print(colorama.Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + colorama.Style.RESET_ALL)
chat()


Start messaging with the bot (type quit to stop)!
User: hi
1/1 [==============================] - 0s 466ms/step
DiamondChatBot: Hello
User: hello
1/1 [==============================] - 0s 25ms/step
DiamondChatBot: Hello
User: what is your name
1/1 [==============================] - 0s 27ms/step
DiamondChatBot: Please call me as Diamond
User: cool! defin deep learnin
1/1 [==============================] - 0s 27ms/step
DiamondChatBot: Please visit https://moodle.lsus.edu/pluginfile.php/1934640/mod_resource/content/1/NatureDeepReview.pdf for more information
User: how about sigmoid?
1/1 [==============================] - 0s 30ms/step
DiamondChatBot: The logistic function, also known as sigmoid, is an activation function commonly used in
 machine learning and artificial neural networks. It transforms any input value to a 
value between 0 and 1. The mathematical formula for the sigmoid function is f(x) = 1 / (1 + e^-x). 
The sigmoid function has an S-shaped curve, and its output increases s